In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import pickle
import multiprocessing as mp
import pandas as pd
from sklearn import datasets
import quante
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern
import time
import warnings
import numpy as np



In [ ]:
iris = datasets.load_iris()
pd.DataFrame(iris.data).to_csv('X.csv', index=False)
with open('y.csv', 'w') as f:
    f.write(','.join([str(x) for x in iris.target]))

In [ ]:
if __name__ == '__main__':
    
    def instance(arch):
    
        X = pd.read_csv('X.csv')
        with open('y.csv') as f:
            y = f.read().split(',')
            
        clf = MLPClassifier(solver='sgd', max_iter=500,
                        hidden_layer_sizes=arch[:3], alpha=arch[3], random_state=1)
        with warnings.catch_warnings(action='ignore'):
            cvs = cross_val_score(clf, X, y)
        return cvs
        
#    print("Call a single instance of MLP Classifier") 
#    print(instance((3,3)))
    
    hp_tune = quante.carlo(instance, [[2, 16], [2, 4], [2, 16], [.01, .99]], kernel=DotProduct()+ WhiteKernel(), n_batches=100, n_processors=4, n_iterations=5)
    
    p = mp.Pool()
    start = time.time()
    
    session, iterations = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()




In [ ]:
summary = session.summary()
summary['iteration_id'] = iterations
summary

In [ ]:
n = summary[['score', 'iteration_id']].groupby('iteration_id').mean().plot()


In [ ]:
rounded_alpha=  [x[:3]+(round(x[3],3),) for x in summary['layers']]

In [ ]:

fig = plt.figure(figsize =(13, 6))
ax = fig.add_subplot(111)
ax.boxplot(session.score_history)
ax.set_xticklabels(rounded_alpha)
plt.xticks(rotation=60)
bx = plt.show


In [ ]:
if __name__ == '__main__':
    
    def instance(hp_parameter):
    
        X = pd.read_csv('X.csv')
        with open('y.csv') as f:
            y = f.read().split(',')
            
        clf = LogisticRegression(solver='saga', penalty = 'elasticnet', l1_ratio=hp_parameter[0], max_iter=200 )
        with warnings.catch_warnings(action='ignore'):
            cvs = cross_val_score(clf, X, y)
        
        return cvs
        
    
    hp_tune = quante.carlo(instance, [[.05, .95]], kernel=DotProduct()+ WhiteKernel(), n_batches=7, n_processors=4, n_iterations=5)
    p = mp.Pool()
    start = time.time()
    
    session, iterations = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()


In [ ]:
if __name__ == '__main__':
    
    def instance(hp_parameter):
    
        X = pd.read_csv('X.csv')
        with open('y.csv') as f:
            y = f.read().split(',')
            
        clf = LogisticRegression(solver='saga', penalty = 'elasticnet', l1_ratio=hp_parameter[0], max_iter=200 )
        with warnings.catch_warnings(action='ignore'):
            cvs = cross_val_score(clf, X, y)
        
        return cvs
        
    
    hp_tune = quante.carlo(instance, [[.05, .95]], kernel=DotProduct()+ WhiteKernel(), n_batches=7, n_processors=4, n_iterations=5)
    p = mp.Pool()
    start = time.time()
    
    session, iterations = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()


In [ ]:
summary = session.summary()
summary['iteration_id'] = iterations
summary

In [ ]:
n = summary[['score', 'iteration_id']].groupby('iteration_id').mean().plot()


In [ ]:
import torch
from torch import nn

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, n_inputs=None, arch=None, n_outputs=None):
        super().__init__()
        self.flatten = nn.Flatten()
        if n_inputs and arch and n_outputs:
            self.make_arch(n_inputs, arch, n_outputs)
                
    def forward(self, x):
        x = self.flatten(x)
        logits = self.stack(x)
        return logits

    def make_arch(self, n_inputs, arch, n_outputs, inner_function='Relu', last_function = None):
        layers = [torch.nn.Linear(n_inputs, arch[0])]
        for a in range(len(arch)-1):
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Linear(arch[a], arch[a+1]))
        layers.append(torch.nn.ReLU())
        layers.append(torch.nn.Linear(arch[-1], n_outputs))
    #    layers.append(torch.nn.Softmax(dim=1))                    # works better without this

        self.stack = torch.nn.Sequential(*layers)

In [ ]:
class data_loader:
    def __init__(self,batch_pct):
        self.X_train = pd.read_csv('X_train.csv')
        self.X_test = pd.read_csv('X_test.csv')
        self.y_train = pd.read_csv('y_train.csv')
        self.y_test = pd.read_csv('y_test.csv')
    
        self.train_batch_size = int(np.floor(self.X_train.shape[0] * batch_pct))
        self.test_batch_size = int(np.floor(self.X_test.shape[0] * batch_pct))
        self.batch_pct = batch_pct
    def get_batch(self,batch):
        if batch < 0:
            return self.X_test, self.y_test
        else:
            return self.X_train[(batch*self.train_batch_size):((batch+1)*self.train_batch_size)],\
                   self.y_train[(batch*self.train_batch_size):((batch+1)*self.train_batch_size)]
                

In [ ]:
if __name__ == '__main__':

    train_batch_size = 1000
    n_batches = 4
    n_iterations = 10

    def instance(arch):

        loss_fn = torch.nn.CrossEntropyLoss()
            
        model = NeuralNetwork(arch[0], arch[1:4], n_outputs=arch[4])
        model.to('cuda:0')
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
        loss_history = []
        model.train()

        loader = data_loader(.05)
    
        for i in range(n_iterations):
            for batch in range(n_batches):
                X_train, y_train = loader.get_batch(batch)

                X_train_torch = torch.tensor(X_train.values, dtype=torch.float)
                y_train_torch = torch.tensor(y_train.values, dtype=torch.float)

                X = X_train_torch.to('cuda:0')
                y = y_train_torch.to('cuda:0')

                pred = model(X)
                loss = loss_fn(pred, y)
        
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss_history.append(loss.item())

        X_test, y_test = loader.get_batch(-1)

        model.eval()
        X_test_torch = torch.tensor(X_test.values, dtype=torch.float)
        y_test_torch = torch.tensor(y_test.values, dtype=torch.float)

        X = X_test_torch.to('cuda:0')
        y = y_test_torch.to('cuda:0')
        pred = model(X)


        return loss.item()#.detatch()



    nn = instance((28*28, 128, 128, 64, 10))


In [ ]:

    hp_tune = quante.carlo(instance, [[.05, .95]], kernel=DotProduct()+ WhiteKernel(), n_batches=7, n_processors=4, n_iterations=5)
    p = mp.Pool()
    start = time.time()
    
    session, iterations = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()
